In [1]:
from netin.models import DPAHModel

In [ ]:
#TODO: check icm diffusion model, error when generating the random values, always reusing the same ones over and over again!!!!
#TODO: reduce seed size, to have more variability in the results

In [2]:
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set
import networkx as nx


from src.seed import (seed_degree, 
    seed_polarized, 
    seed_random, 
    seed_polarized_centrality_mixed, 
    seed_polarized_centrality, 
    seed_polarized_degree, 
    seed_centrality_mixed, 
    seed_centrality, 
    seed_influence_maximization
)


from evaluation import evaluate_all_seeds

This model connects a new node to the existing nodes biased by their group assignment. Homophily values of $h > 0.5$ indicate homophily, meaning that nodes prefer to connect to their own group. Heterophily is specified by $h < 0.5$. Nodes tend to connect to the other group.

Instead of the number of links per each new node, the directed graph models parameterize the density of the final network by the parameter $d$. Additionally, we need to define activity levels for outgoing links for both the minority and majority group as $plo_m/M$

In [3]:
N = 10_000 # Number of total nodes
f_m = 0.3 # Fraction of minority nodes
h = 0.5 # Homophilic
d = 0.001 # desity of the graph
plo = 2.0
#final number of edges will be: e = d * n (n-1)

In [4]:
# DPAH graphs:
# Homophilic h > 0.5
# Neutral h = 0.5
# Heterophilic h < 0.5

homophily_values = [0.1, 0.5, 0.9]

In [5]:
dpa_model = DPAHModel(N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h)
dpa_graph = dpa_model.simulate()

In [6]:
G = dpa_graph.to_nxgraph() # This creates a copy of the graph

for node, attr in G.nodes(data=True):
    attr['color'] = attr.pop('minority')

for node in G.nodes():
        in_degree = G.in_degree(node)
        for neighbor in G.successors(node):
            G[node][neighbor]["weight"] = 1 / in_degree if in_degree > 0 else 1

In [7]:
seed_size = G.number_of_nodes() // 20
print(f"Seed size: {seed_size}")
k = seed_size
max_iter = 1000
budget = 3 * seed_size

Seed size: 500


In [8]:
# Define your seed functions
seed_functions = {
    "Random": seed_random,
    "MIA": lambda G, seed_size: seed_influence_maximization(G, seed_size, 200, 8, 1),
    "Degree": seed_degree,
    "Centrality": seed_centrality,
    "Centrality Mixed": seed_centrality_mixed,
    "Polarized": lambda G, seed_size: seed_polarized(G, seed_size, 0),
    "Polarized Degree": lambda G, seed_size: seed_polarized_degree(G, seed_size, 0),
    "Polarized Centrality": lambda G, seed_size: seed_polarized_centrality(G, seed_size, 0),
    "Polarized Centrality Mixed": lambda G, seed_size: seed_polarized_centrality_mixed(G, seed_size, 0),
}

# Example of how to call the function
all_results_df = evaluate_all_seeds(G, seed_functions, seed_size, k, max_iter, budget, "netin_05")

# Display the combined results
print("Combined Results for All Seed Functions")
all_results_df

Running evaluation for seed function: Random
    Running evaluation for graph modification: Original Graph


100%|██████████| 1000/1000 [00:18<00:00, 52.74it/s]


    Running evaluation for graph modification: Random


100%|██████████| 1000/1000 [00:25<00:00, 39.58it/s]


    Running evaluation for graph modification: Degree


...

100%|██████████| 1000/1000 [00:45<00:00, 22.18it/s]


    Running evaluation for graph modification: Custom V2


100%|██████████| 1000/1000 [00:44<00:00, 22.28it/s]

Combined Results for All Seed Functions


Metric,Seed Function,Graph Modification,Avg Activated Nodes,Activated Nodes Std Dev,"Avg Activated Nodes, Color (0)","Activated Nodes Std Dev, Color (0)","Avg Activated Nodes, Color (1)","Activated Nodes Std Dev, Color (1)",Avg Color Activation Count,Color Activation Count Std Dev,Number of Nodes,Number of Edges
0,Centrality,Original Graph,967.546,149.242,675.337,104.137,292.209,46.453,189.069,67.136,10000.0,99990.0
1,Centrality,PrefAtt,9110.000,0.000,6370.000,0.000,2740.000,0.000,18753.205,5762.145,10000.0,101490.0
2,Centrality,Jaccard,9110.000,0.000,6370.000,0.000,2740.000,0.000,14567.275,2844.212,10000.0,101490.0
3,Centrality,Degree,9110.000,0.000,6370.000,0.000,2740.000,0.000,6781.539,174.222,10000.0,101490.0
4,Centrality,TopK,9110.000,0.000,6370.000,0.000,2740.000,0.000,21344.383,6417.371,10000.0,101490.0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Random,TopK,9170.000,0.000,6412.000,0.000,2758.000,0.000,15868.487,2893.623,10000.0,101490.0
77,Random,KKT,9170.000,0.000,6412.000,0.000,2758.000,0.000,15990.690,2903.314,10000.0,101490.0
78,Random,Random,1929.672,200.336,1338.078,141.302,591.594,60.486,960.122,212.220,10000.0,101490.0
79,Random,Custom,1947.790,243.373,1179.496,156.400,768.294,95.062,1335.915,336.820,10000.0,101490.0


In [9]:
all_results_df.to_csv(f"results/results_netin_{N}_nodes_{h}_hvalue_{d}_density_{plo}_plo.csv", index=False)